In [2]:
import pyrootutils
import hydra
import torch

root = pyrootutils.setup_root(search_from="./", indicator=[".git", "pyproject.toml"], pythonpath=True, dotenv=True)


In [3]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize("../configs/experiment/", version_base="1.2")
model_config = hydra.compose("mae3d.yaml")

ConfigCompositionException: In 'mae3d.yaml': Could not override 'datamodule'. No match in the defaults list.

In [3]:
model_config

{'optimizer': {'_target_': 'torch.optim.SGD', '_partial_': True, 'lr': 0.1, 'weight_decay': 0.0001, 'momentum': 0.9}, 'scheduler': {'_target_': 'torch.optim.lr_scheduler.OneCycleLR', '_partial_': True, 'max_lr': 0.5, 'epochs': '${trainer.max_epochs}'}, 'model': {'net': {'_target_': 'src.common.nn_modules.nets.classifiers.qnet.Qnet', 'num_classes': 2, 'encoder_channels': [1, 16, 32, 64, 128], 'return_feats': False}}, '_target_': 'src.models.combined_module.CombinedLitModule', 'activation': {'_target_': 'torch.nn.Softmax', 'dim': 1}, 'metric': {'_target_': 'src.metrics.classification.class_metrics.ClassificationMetrics', 'num_classes': 2, 'ignore_index': -100}, 'criterion': {'_target_': 'src.common.nn_modules.losses.cross_entropy_loss.CrossEntropyLossFunc', 'ignore_index': -100, 'weights': {'_target_': 'torch.tensor', 'data': [1.0, 3.0]}}, 'additional_keys': None, 'subgroups_keys': None, 'image_key': 'image', 'label_keys': 'label'}

In [4]:
model_config['model']['net']

{'_target_': 'src.common.nn_modules.nets.classifiers.qnet.Qnet', 'num_classes': 2, 'encoder_channels': [1, 16, 32, 64, 128], 'return_feats': False}

In [4]:
model = hydra.utils.instantiate(model_config['model']['net'])

/home/users/shubham.kumar/miniconda3/envs/qct_deep_clone/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


InstantiationException: Error locating target 'src.common.nn_modules.nets.classifiers.qnet.Qnet', see chained exception above.
full_key: model.net

In [6]:
ckpt_path = "/data_nas5/qer/shubham/fpr_sl_checkpoints/fpr_sl/runs/2023-12-11_20-07-27/checkpoints/epoch=28_step=667_val_loss=0.09.ckpt"



In [7]:
from src.common.nn_modules.nets.segmentation.multitaskfusion import modify_model_state_dict

In [8]:
k= torch.load(ckpt_path, map_location = "cpu")
state_dict = k['state_dict']
new_state_dict = modify_model_state_dict(state_dict , "net." , "")

model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [9]:
model = model.to("cuda:1")

In [10]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize("../configs/datamodule/", version_base="1.2")
data_module = hydra.compose("fpr_mae_datamodule.yaml")

In [11]:
data_module

{'transforms': {'val_transforms': {'_target_': 'monai.transforms.Compose', 'transforms': [{'_target_': 'src.common.transforms.monai_extras.LoadSafeTensor', 'image_key': 'image'}, {'_target_': 'src.common.transforms.monai_extras.ApplyWindowsChannelWise', 'keys': ['image'], 'windows': ['LungA']}]}, 'train_transforms': {'_target_': 'monai.transforms.Compose', 'transforms': [{'_target_': 'src.common.transforms.monai_extras.LoadSafeTensor', 'image_key': 'image'}, {'_target_': 'src.common.transforms.monai_extras.ApplyWindowsChannelWise', 'keys': ['image'], 'windows': ['LungA']}, {'_target_': 'src.common.transforms.monai_extras.RandomHorizontalFlip', 'keys': ['image'], 'prob': 0.5}, {'_target_': 'src.common.transforms.monai_extras.RandomGaussianBlur', 'keys': ['image'], 'kernel_size': 3, 'sigma': [0.1, 2.0], 'prob': 0.5}]}}, '_target_': 'src.datamodules.base_dataloader.MetaClsDataModule', 'data_cfg': {'data_extract_func': 'videomae_data_loader', 'csvs': ['/home/users/shubham.kumar/projects/qc

In [12]:
datamodule = hydra.utils.instantiate(data_module)

Unable to find logging config yaml. Configuring fallback values
[2023-12-14 16:44:31 +0530] [WARNING] Invalid or unsupported selector 'select:not(:has(option[selected])) option:first-of-type,
option[selected]:not(option[selected] ~ option[selected]) ', ('Unknown pseudo-class', 'has')


2023-12-14 16:44:33.467 | INFO     | src.datamodules.base_dataloader:setup:56 - Train set Length: 4715
2023-12-14 16:44:33.469 | INFO     | src.datamodules.base_dataloader:setup:57 - Val set Length: 834
2023-12-14 16:44:33.469 | INFO     | src.datamodules.base_dataloader:setup:64 - Test set Length: 0


In [13]:
train_ds = datamodule.train_ds
val_ds = datamodule.val_ds
# test_ds = datamodule.test_ds

In [14]:
from torch.utils.data import DataLoader

In [15]:
dataloader = DataLoader(
    train_ds,
    batch_size= 32,
    shuffle=False,
    num_workers=16,
    drop_last=True,
    pin_memory=True,
)


In [18]:
activation = torch.nn.Softmax(dim = 1)

In [20]:
result = []
for batch in dataloader :
    image = batch["image"].to("cuda:1")
    output = activation(model.forward(image))[:,0].detach().cpu1()
    result.append(output)

In [21]:
final_out = torch.concat(result)

In [22]:
final_out.shape

torch.Size([4704])

In [23]:
len(final_out[final_out > 0.5])

3033

In [65]:
len(final_out[final_out < 0.5])

514107

In [49]:
specificity = 116381/117952

In [50]:
specificity

0.9866810227889311

In [58]:
specificity = len(final_out[final_out < 0.5])/final_out.shape[0]
specificity

0.9849509947337625

In [66]:
specificity = len(final_out[final_out < 0.5])/final_out.shape[0]
specificity

0.9744552526232789

In [31]:
sensitivity = len(final_out[final_out > 0.1])/final_out.shape[0]
sensitivity

0.7908163265306123